In [ ]:
import tensorflow as tf 
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.python.keras.utils import generic_utils
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
DataDirectory="C:\Mihir\Projects\Emotion Detector+Movie Recommendation+Review Analysis\Image Dataset\Train"
Classes=["Happy","Sad","Scared"]

training_Data= []

img_size=224

def Create_Training_Data():
    for category in Classes:
        path=os.path.join(DataDirectory,category)
        class_num=Classes.index(category)  #0,1,2....Labels
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass


Create_Training_Data()

import random
random.shuffle(training_Data)

X=[] #A list
y=[] #A list

for features,label in training_Data:
    X.append(features)
    y.append(label)

X=np.array(X).reshape(-1,img_size,img_size,3)  #Design Matrix size (shape= #no.of.images x pixels x pixels x 3)  
                                               #We put-1 so as to automatically detect the number of images

#Normalize the data
X=X/255.0 #Other way to normalize=Using scikit learn

Y=np.array(y) #Label Matrix (shape= #no.of.images x 1 )

#X.shape

In [ ]:
model=tf.keras.applications.MobileNetV2() #Pretriained model
model.summary()

In [ ]:
#Remove/Modify some things in the huge model
base_input=model.layers[0].input   #'0' means that for input we are taking the first layer of the model
base_output=model.layers[-2].output #Discard the last layer i.e. -1 from the Pretrained model...start from -2

final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output= layers.Dense(3,activation='softmax')(final_output)  #3 classes

In [ ]:
new_model=keras.Model(inputs=base_input,outputs=final_output)
new_model.summary()

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])  #Settings for multi-class classification

In [ ]:
new_model.fit(X,Y,epochs=15)

In [ ]:
new_model.save('emotion_detector.h5')

In [ ]:
#Testing the Model   trained_model.evaluate #For getting metrics on test data set